#  Submit test

In [ ]:
%aiida
# General imports
import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from ase.data import vdw_radii
from ase import Atoms
from IPython.display import display, clear_output, HTML
import itertools

from traitlets import dlink, Dict

# AiiDA imports
from aiida_cp2k.utils import Cp2kInput
from aiida.orm import SinglefileData
from aiidalab_widgets_base import CodeDropdown, SubmitButtonWidget,  StructureBrowserWidget
from aiidalab_widgets_base.utils import string_range_to_set
from aiida_cp2k.workchains.base import Cp2kBaseWorkChain

# Custom imports

from apps.surfaces.widgets.ANALYZE_structure import StructureAnalyzer
from apps.surfaces.widgets.cp2k2dict import CP2K2DICT
from apps.surfaces.widgets.create_xyz_input_files import make_geom_file
from apps.surfaces.widgets.inputs import PlainInputDetails, InputDetails
from apps.surfaces.widgets.viewer_details import ViewerDetails
from apps.surfaces.widgets.slab_validity import slab_is_valid
from apps.surfaces.widgets.cp2k_input_validity import validate_input #input_is_valid
from apps.surfaces.widgets.suggested_param import suggested_parameters
from apps.surfaces.widgets.get_cp2k_input import Get_CP2K_Input

from aiidalab_widgets_base import CodeDropdown, StructureManagerWidget, StructureBrowserWidget, StructureUploadWidget, SubmitButtonWidget, SmilesWidget

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
## STRUCTURE SELECTOR
structure_selector = StructureManagerWidget(importers=[
    ("AiiDA database", StructureBrowserWidget()),
    ("Import from computer", StructureUploadWidget()),
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

In [ ]:
## DESCRIPTION OF CALCULATION
process_description = ipw.Text(description='Process description: ',
                               placeholder='Type the name here.',
                               style={'description_width': '120px'},
                               layout={'width': '70%'})

In [ ]:
output = ipw.Output()
def update_dict(_=None):
    with output:
        clear_output()
        print(input_details.return_final_dictionary())
display_updated = ipw.Button(description="display updated")
display_updated.on_click(update_dict)

In [ ]:
# Code
computer_code_dropdown = CodeDropdown(input_plugin='cp2k')
input_details = InputDetails()
input_details.observe(update_dict, names='details')
analyzer = StructureAnalyzer()
dlink((structure_selector, 'structure'), (analyzer, 'structure'))
dlink((analyzer, 'details'), (input_details, 'details'))
display(input_details)

In [ ]:
## DISPLAY SUBMIT BUTTON
btn_submit = SubmitButtonWidget(process=Cp2kBaseWorkChain, input_dictionary_function=None)
display(btn_submit, display_updated, output)